In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [48]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [27]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
    
# get characters that appear in text    
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(f"vocabulary_size: {vocab_size}")

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
vocabulary_size: 81


In [28]:
# tokenizer
# map from str to int and int to str
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [29]:
data = torch.tensor(encode(text), dtype=torch.long) 
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


In [30]:
# train and validation split
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [51]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch("train")
print(f"Inputs: \n{x}")
print(f"Targets: \n{y}")

Inputs: 
tensor([[72,  1, 72, 61, 58,  1, 65, 68],
        [78, 68, 74,  1, 56, 54, 65, 65],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [74, 58, 58, 71,  1, 65, 62, 60]], device='cuda:0')
Targets: 
tensor([[ 1, 72, 61, 58,  1, 65, 68, 68],
        [68, 74,  1, 56, 54, 65, 65,  1],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [58, 58, 71,  1, 65, 62, 60, 61]], device='cuda:0')


In [21]:
# inputs and targets
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, target is {target}")

when input is tensor([80]), target is 0
when input is tensor([80,  0]), target is 1
when input is tensor([80,  0,  1]), target is 1
when input is tensor([80,  0,  1,  1]), target is 28
when input is tensor([80,  0,  1,  1, 28]), target is 39
when input is tensor([80,  0,  1,  1, 28, 39]), target is 42
when input is tensor([80,  0,  1,  1, 28, 39, 42]), target is 39
when input is tensor([80,  0,  1,  1, 28, 39, 42, 39]), target is 44


In [41]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, index, targets=None):
        logits = self.token_embeding_table(index)

        if targets is None:
            loss = None
        else:
            # batch, time, channel
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of idices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B,T+1)
            
        return index

In [44]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


AUn9NQFo
PI[3[z&'kM5f_4mE7n)o3E'eFjxh.Pm,9QL'U1?wSqG2Ab7CMBw,Eq
cct1c_tERh9I'if;v-ehzA.7Fp-N ,iR2H3vukKprh:A﻿F-47t?ajg?gR"no
ND
aFzLxa' ?(MEZ?Tt09iPU_]Xex8pqGOa_
L](ggRBf94MjhURZjDp9HSf&O'I-;v(kv3[JGl4-&KBSjO_DzxEXwBqNURe2dE1D[l_.7_zfTG;﻿WKRVOxKOhW﻿2VOtvk-g9a)'o7npg9kpmUM] 4MX?9HS*hl9n"ns6qogiW90n9CpPd8*jN tonA0s5rR;F T[vv r&"s5VY7:&G5grhMEWA;H3z:dgVY2﻿g :5V;gRZex-XL]M﻿4BabnBRMB3L2v?&gRe]qvl "J4-a;IRe"nAYbTYgJYh]mCL""*JZ
jF'i? ;8,9(QsX)K7T[n7n?_3Kebqj,aV[zxeiC)[Dipe:ApmIenk"&,dbYJ4Ttxqvl7pq:﻿qVd


In [55]:
# create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# training loop
for iter in range(max_iters):
    # sample batch of data
    xb, yb =  get_batch("train")

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.3027405738830566


In [56]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



croo haswh thewe mu'st trpy, ane wait abin Oheng as ouse ind tasiggine.

"Whe fattt o s thie rre gudndive ftopofrea, yofe hy, lineceoft cor.

ta a thorth

awirmoug BAPatind wher Jire'sarishe be baimed:comut thored,

re te hy inyoige 'swe t onscok rclig un t wean wf thecuthkeanoo fr; ce ing bors te  f idrd heirar bo aimofr hem. cace alld at bl spor ITharevewe ms ve S.

Doknsporewhede cose beakeno ld maraiverin, d am res ngyodeling wacemathe G_kanse n l o mey tsce adizaf try. PGOOZey mepe allousc
